In [1]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time

In [2]:
driver = webdriver.Chrome('data/chromedriver')

In [3]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
driver.get(url)
time.sleep(10)
body = driver.find_element_by_css_selector('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.4)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.159)


In [ ]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [ ]:
# 문자 숫자 int 형 바꾸자
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억','').replace('개','').replace(',','')
    s = s.replace('만','0000')

    return f'{int(s):,d}'

In [ ]:
# pop-up 창이 있으면, pop-up 창을 닫고 메인 윈도우로 되돌아 옴
window_len = len(driver.window_handles)
if window_len == 2:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()

# 화면의 끝까지 PageDown으로 내려보냄
driver.switch_to.window(driver.window_handles[0])
body = driver.find_element_by_css_selector('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [ ]:
channels =[]
# cat_list, name_list, sub_list, view_list, video_list =[], [], [], [], []
for channel in trs:
    cat = channel.find_element_by_css_selector('.category').text.strip(' \n[]')
    name = channel.find_element_by_css_selector('.subject a').text.strip()
    sub = channel.find_element_by_css_selector('.subscriber_cnt').text
    view = channel.find_element_by_css_selector('.view_cnt').text
    video = channel.find_element_by_css_selector('.video_cnt').text[:-1] #개 삭제

    channels.append([cat, name, sub, view, video])

In [ ]:
df = pd.DataFrame(channels, columns=['카테고리', '채널명', '구독자수', '조회수', '영상수'])
df

,카테고리,채널명,구독자수,조회수,영상수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223
...,...,...,...,...,...
95,음식/요리/레시피,까니짱 [ G-NI ],408만,10억3471만,482
96,키즈/어린이,로미유 스토리[Romiyu Story],408만,10억6621만,433
97,음악/댄스/가수,NCT DREAM,387만,3억7615만,229
98,BJ/인물/연예인,허팝Heopop,381만,31억3892만,"1,847"


In [ ]:
results = []
for page in range(1,11):
    #print(page)
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)

    window_len = len(driver.window_handles)
    if window_len == 2:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    trs = driver.find_elements_by_css_selector('.aos-init')

    for tr in trs:
        category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
        #category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = tr.find_element_by_css_selector('.subject a').text.strip()
        #name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
        #subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
        #view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)
        #video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [ ]:

df = pd.DataFrame(results, columns=['카테고리','채널명','구독자수','조회수','비디오수'])
df

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,000,000","19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,"60,300,000","18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223
...,...,...,...,...,...
995,미분류,밉지않은 관종언니,"510,000","67,950,000",170
996,게임,미소,"510,000","210,780,000","3,084"
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"


In [ ]:
driver.close()